In [1]:
import sys
sys.path.append("..")

from turbochat.v1.prompt import GPTMsges, GPTToolPrompt
from turbochat.v1.gpt import GPT, Msg, Tool
from Apollo.Conversation.converse.prompts.user import PromptMaker
from Apollo.store.mongoengine import MongoHistoryWithFAISS

import pandas as pd
import json
from mongoengine import connect

from Apollo.Conversation.serializers import *


GPT_API_KEY_PATH = r"C:\Users\nagen\Desktop\gpt\project_apollo\Apollo\openai_key"

C:\Users\nagen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\nagen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarnin

In [8]:
MONGO_INSTANCE = connect(db='Apollo', host="mongodb://localhost:27017/")

# gpt api key
with open(GPT_API_KEY_PATH, 'r') as f:
    GPT_KEY = f.read()
    gpt = GPT(GPT_KEY, model="gpt-4o")

### tool definition

In [48]:
tool_mode_selector = {
    "type": "function",
    "function": {
        "name": "detect_conversation_intent_mode",
        "description": """
        Set the conversation mode to a specific mode. 
        Conversation mode can be set to different modes like: 
        - normal: Normal conversation mode, where user and assistant can talk normally.
        - appointment_or_service_purchase: Conversation mode where user is setting an appointment or purchasing a service.
        - goal: Conversation mode where user is setting a goal.
        - advice: Conversation mode where user is asking for advice.
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "description": """
                    conversation mode to set:
                    
                    - normal: 
                        Default conversation mode where the user speaks normaly
                    
                    - appointment_or_service_purchase: 
                        Detects if user wants or has intended to set an appointment or purchase a service from the user's recent conversation. 
                    
                    - goal: 
                        Detects if user wants or has intended to set a goal from the user's recent conversation. 
                        Goals can be related to tracking, managing, or aiming for some health, fitness, lifestyle, mental health, etc related target.
                        For example: If user says something like I want to keep track of my Blood Pressure, then that would be considered as a goal.
                    
                    - advice: 
                        Detects if user is asking for advice from the user's recent conversation. 
                        Asking for advice means user is looking for some user specific suggestions, summarization, or recommendations.
                        For example: If user says something like I need advice on how to reduce my stress level, then that would be considered as a goal.
                    
                    """,
                    "enum": ["normal", "appointment_or_service_purchase", "goal", "advice"]
                }
            },
            "required": ["mode"]
        }
    }
}


tool_appointment_or_purchase_service = {
    "type": "function",
    "function": {
        "name": "set_an_appointment_or_purchase_service",
        "description": """
            Use this funtion only if the user has requested to set an appointment with a specific doctor or has requsted to purchase a service package. 
            By looking at the recent conversation history, if the user has requested to set an appointment with the doctor 
            or has requested to purchase a service package, this tool will trigger.
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "event_type": {
                    "type": "string",
                    "enum": ["appointment", "service_purchase"],
                    "description": "Type of event requested by the user",
                },

                "event_description": {
                    "type": "string",
                    "description": """
                    description of the event. 
                    For example, 'Requested appointment with Dr. ABC' or 'Requested to purchase a XYZ package' etc
                    """,
                },

                "event_contact": {
                    "type": "string",
                    "description": "Name of the Doctor or service provider company",
                },

                "event_date": {
                    "type": "string",
                    "description": "Scheduled date of the event. What is the requested appointment date or purchase date. In Python datetime Format: %Y-%m-%d",
                },

                "event_time": {
                    "type": "string",
                    "description": "Scheduled time of the event. What is the requested appointment time or purchase time. in Python datetime Format: %H:%M:%S",
                },

            },
            "required": [
                "event_type",
                "event_description",
                "event_contact",
                "event_date",
                "event_time",
            ],
        },
    }
}


tool_extract_goal_details = {
    "type": "function",
    "function": {
        "name": "Extract_goal_details",
        "description": """
            This tool sets the goal request from the user's recent conversation history.
            By looking at only the user's recent conversation and context, it extracts important key value pairs:

            - goal_type: Type of goal requested by the user
            - goal_description: description of the goal
            - goal_milestones: Milestones of the goal
            - goal_progress: Has there been some goal progress already?
            - goal_target_date: Target date of the goal

        """,
        "parameters": {
            "type": "object",
            "properties": {
                "goal_type": {
                    "type": "string",
                    "enum": ["health", "fitness", "lifestyle", "mental_health"],
                    "description": "Type of goal requested by the user",
                },

                "goal_description": {
                    "type": "string",
                    "description": """
                    description of the goal. 
                    For example, 'Want to reduce weight by 10 pounds' or 'Want to run 5 miles daily' etc
                    """,
                },

                "goal_milestones": {
                    "type": "array",
                    "items": {
                        "type": "string",
                    },
                    "description": "Milestones of the goal. What are the steps or milestones to achieve the goal",
                },

                "goal_progress": {
                    "type": "number",
                    "description": """
                    Has there been some goal progress already? If yes, what is the progress in percentage.
                    0 means no progress and 100 means goal is achieved.
                    Example: if user has already lost 2 pounds out of 10 pounds, then progress would be 20%
                    """,
                },

                "goal_target_date": {
                    "type": "string",
                    "description": "Target date of the goal. When does the user want to achieve the goal. In Python datetime Format: %Y-%m-%d",
                },

            },
            "required": [
                "goal_type",
                "goal_description",
                "goal_milestones",
                "goal_progress",
                "goal_target_date",
            ],
        },
    }
}


TOOL_MODE_SELECTOR = GPTToolPrompt(tool_mode_selector)


TOOL_EXTRACT_GOAL_DETAILS = GPTToolPrompt(tool_extract_goal_details)


TOOL_APPOINTMENT_OR_PURCHASE_SERVICE = GPTToolPrompt(tool_appointment_or_purchase_service)


### tool callable

In [49]:

def MODE_SELECTOR_CALL(mode):
    return mode


def EXTRACT_GOAL_DETAILS_CALL(goal_type, goal_description, goal_milestones, goal_progress, goal_target_date):
    return {"goal_type": goal_type, "goal_description": goal_description, "goal_milestones": goal_milestones, "goal_progress": goal_progress, "goal_target_date": goal_target_date}


def APPOINTMENT_OR_PURCHASE_SERVICE_CALL(event_type,event_description,event_contact,event_date,event_time):
    return {"event_type": event_type, "i_event_description": event_description, "event_contact": event_contact, "event_date": event_date, "event_time": event_time}


In [43]:
SYSTEM_PROMPT = """
You are role playing as a healthcare ASSISTANT.
With every prompt from the USER, a relevent context we already know about the USER will be appended to the prompt.
The context will include user's health history, goals, services and doctors. Use the context if relevent to guide the conversation.
You will also be provided with some tools. If the the user wants to to set an appointment or purchase service, use the tools to extract the data and request the event.
Stick to YOUR GOAL and never disobey YOUR STRICT RULES.

YOUR PERSONALITY:
You collect as much information about the user as possible by asking QUESTIONS and chaining conversation.
If conversation goes off topic, acknowledge the user's responses and get the conversation back to health and lifestyle.
Suggest small goals, doctors, or relevent service packages from provided context if relevent in order to help with their problem.

YOUR GOAL:
Collect information regarding everything which is an indicator of a user's health.
For example: their habits, mental, physical health, medical conditions, reports, tests, lifestyle choices preferences etc. 

YOUR STRICT RULES:
You strictly follow these rules:
- If the user asks to purchase a service, always acklowledge the request and say that you have requested it.
- You only talk about one specific health indicator at a time.
- You must respond in short sentences. (1-3 lines maximum)
- You must act caring, helpful and friendly.
- Keep the tone of the conversation casual.
- ALWAYS consider the user's past history while collecting user's information.

You have the ability to use tools.
Use detect_conversation_intent_mode tool to detect conversation intent from the user so that the system can help you make the response.
"""

# You have the ability to use tools. 
# Use trigger_intent_appointment_or_purchase_service tool if the user intends to set an appointment or purchase a service.
# Use trigger_if_user_intends_set_a_goal tool if the user is trying to tracking, managing, or aiming for some health related target.
# Use trigger_if_user_is_asking_for_advice tool if the user is asking for personalized understanding, summary, or advice.

In [50]:
# MESSAGE = "I want to make an appointment with the doctor"
MESSAGE = "how do i manage my bmi? I need recommendations."
# MESSAGE = "I want to summarise my health history"
# MESSAGE = "Hi how are you"

In [51]:
# default context

USER_ID = 7

# RAG Key value pairs
faiss_history = MongoHistoryWithFAISS(
    USER_ID,
    MONGO_INSTANCE,
    ConversationHistoryWithFaissSupportSchema, 
    ConversationHistoryWithFaissSupportSchemaSerializer
)
history_data = faiss_history.get(MESSAGE, k=10)


# normal
context = {
    "history": json.dumps(history_data.to_dict("records"), indent=2) if not history_data.empty else "",
    "user_prompt": MESSAGE
}

C:\Users\nagen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [36]:
# when selecting mode ["normal", "appointment_or_service_purchase", "goal", "advice"]

context_mode_selector = {"user_prompt": context["user_prompt"]}
_context = context_mode_selector

is_system_prompt_required = False

In [28]:
# when, mode = appointment_or_service_purchase

special_instructions_event = """
Ask the user each of the following questions to get the required information about the appointment or service purchase.
- confirm the exact doctor or service package
- confirm the appointment/purchase date and appointment/purchase time
- lastly confirm the appointment/service purchase

You have the ability to use tools. 
Once all the information described above is collected,
Use Extract_appointment_or_purchase_service_details tool to set the appointment/purchase and confirm to the user that you have set it.
"""


# RAG doctors
faiss_doctors = MongoHistoryWithFAISS(
    0,
    MONGO_INSTANCE, 
    DoctorsWithFaissSupportSchema,
    DoctorsWithFaissSupportSchemaSerializer
)
doctors = faiss_doctors.get(MESSAGE, k=5)


# RAG services
faiss_services = MongoHistoryWithFAISS(
    0,
    MONGO_INSTANCE, 
    ServiceWithFaissSupportSchema,
    ServiceWithFaissSupportSchemaSerializer
)
services = faiss_services.get(MESSAGE, k=5)


context_event = context.copy()
context_event["doctors"] = json.dumps(doctors.to_dict("records"), indent=2) if not doctors.empty else "",
context_event["services"] = json.dumps(services.to_dict("records"), indent=2) if not services.empty else "",
context_event['instructions'] = special_instructions_event
_context = context_event

is_system_prompt_required = True

In [46]:
# when, mode = goal

special_instructions_goal = """
Keep asking these questions natruallly in the conversation until you have all these required information about the goal:

- confirm the goal description
- confirm the goal milestones
- Has there been some goal progress already?
- confirm the goal target date

You have the ability to use tools.
Once you have collected all the information described above and the user has confirmed the goal details,
Use Extract_goal_details tool to set the goal and confirm to the user that you have set it.
"""

context_goal = context.copy()
context_goal['instructions'] = special_instructions_goal

is_system_prompt_required = True


### user prompt

In [38]:
pm = PromptMaker({
    "user_prompt": {"label": "user prompt"},
    "services": {"label": "Available services"},
    "doctors": {"label": "Available doctors"},
    "history": {"label": "User related information"},
    "instructions": {"label": "Special system instructions"}
})


messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": pm.get(_context)
            }
        ]
    }
]


if is_system_prompt_required:
    messages = [{
        "role": "system",
        "content": SYSTEM_PROMPT
    }] + messages


TOOL_MESSAGES = GPTMsges(messages)

### Tool Call

In [39]:
# mode selector
tool = Tool(gpt, TOOL_MESSAGES, TOOL_MODE_SELECTOR, MODE_SELECTOR_CALL, tool_choice="required")
response_mode_selector, tool_prompt_mode_selector, results_mode_selector = tool.call()
print(results_mode_selector)

(True, {'detect_conversation_intent_mode': ['advice']})


In [ ]:
# goal extraction
tool = Tool(gpt, TOOL_MESSAGES, TOOL_EXTRACT_GOAL_DETAILS, EXTRACT_GOAL_DETAILS_CALL, tool_choice="auto")
response_extract_goals, tool_prompt_extract_goals, results_extract_goals = tool.call()
print(results_extract_goals)

In [52]:
# appointment_or_service_purchase extraction
tool = Tool(gpt, TOOL_MESSAGES, TOOL_APPOINTMENT_OR_PURCHASE_SERVICE, APPOINTMENT_OR_PURCHASE_SERVICE_CALL, tool_choice="auto")
response_appoint_or_service, tool_prompt_appoint_or_service, results_appoint_or_service = tool.call()
print(results_appoint_or_service)

(False, {})
